In [1]:
import sys
import os

# Add the src directory to the Python path
try:
    current_dir = os.path.dirname(__file__)
except NameError:
    current_dir = os.getcwd()  # Fallback to the current working directory

src_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
sys.path.append(src_dir)

# Now import the module from utils
from src.utils.etl import ETL

Collecting Data:

rating = 1 for every shiur in database. Can make something more advanced later.

In [2]:
db = ETL()
df = db.get_bookmakrs_df()
df['rating'] = 1
df.shape

(24677, 11)

Create fastai DataLoaders

In [3]:
from fastai.collab import *

dls = CollabDataLoaders.from_df(df, user_name='user', item_name='shiur', rating_name='rating', bs=64)
dls.show_batch()

,user,shiur,rating
0,80580.0,1024228,1
1,79212.0,955116,1
2,221467.0,1079301,1
3,92378.0,1099483,1
4,211439.0,1065734,1
5,35049.0,1012650,1
6,209914.0,869348,1
7,218322.0,889236,1
8,212079.0,878883,1
9,10794.0,1098888,1


Run the model

In [4]:
from fastai.vision.all import *

learn = collab_learner(dls, n_factors=5, y_range=(0, 1), loss_func=BCEWithLogitsLossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.459713,0.455195,00:01
1,0.418023,0.417283,00:01
2,0.387775,0.394726,00:01
3,0.376714,0.386109,00:01
4,0.373993,0.384774,00:01


Save the model

In [21]:
learn.model_dir = "saved_models/"
learn.save("user_collab_filtering_v1")

Path('saved_models/user_collab_filtering_v1.pth')

Use model

In [56]:
from typing import Dict, List
from base import BaseModel



class UserCollabFilteringV1(BaseModel):
    def __init__(self):
        model = learn.load("user_collab_filtering_v1")

    def get_recommendations(self, user_id: str = None, *args, **kwargs) -> List[int]:
        top_n = kwargs.get('top_n', 10)     
        all_item_ids = dls.classes['shiur'].items
        user_idx = dls.classes['user'].o2i[user_id]
        
        # Create a list of user-item pairs
        user_item_pairs = [(user_idx, dls.classes['shiur'].o2i[item_id]) for item_id in all_item_ids]
        
        # Create a DataLoader for the user-item pairs
        user_item_dl = dls.test_dl(user_item_pairs)
        
        # Get predictions
        preds, _ = learn.get_preds(dl=user_item_dl)
        
        # Squeeze the predictions to remove extra dimensions
        preds = preds.squeeze()
        
        # Sort the predictions in descending order and get the indices
        sorted_idxs = torch.argsort(preds, descending=True)
        
        # Get the top N item indices
        top_items = [all_item_ids[i] for i in sorted_idxs[:top_n]]
        
        return top_items

    def get_weighted_recommendations(self, user_id: str = None, *args, **kwargs) -> Dict[int, float]:
        return super().get_weighted_recommendations(user_id, *args, **kwargs)
    
    def get_best_shiurim(self, shiur_num:int = 10): #based on highest bias
        shiur_bias = learn.model.i_bias.weight.squeeze()
        idxs = shiur_bias.argsort(descending=True)[:shiur_num]
        return [dls.classes['shiur'][i] for i in idxs]
    
    def get_user_bias(self, user_id:str = None):
        user_biases = learn.model.u_bias.weight
        user_idx = learn.dls.classes['user'].o2i[user_id]
        return user_biases[user_idx]
    
    def get_shiur_bias(self, shiur_id:str = None):
        item_biases = learn.model.i_bias.weight
        item_idx = learn.dls.classes['user'].o2i[shiur_id]
        return item_biases[item_idx]

Testing model

In [57]:
model = UserCollabFilteringV1()
print(model.get_shiur_bias('1098888'))
print(model.get_user_bias('0.387775	'))
print(model.get_best_shiurim(4))
print(model.get_recommendations("79212.0", top_n = 5))

tensor([0.0051], grad_fn=<SelectBackward0>)
tensor([0.0168], grad_fn=<SelectBackward0>)
[1098754, 1098964, 1098495, 1098683]


TypeError: list indices must be integers or slices, not list